In [1]:
import os
import socket
import subprocess
import pandas as pd
import io
from contextlib import redirect_stdout
import itertools
import warnings
from datetime import datetime
import time
import timeout_decorator
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [2]:
def islocal(): return socket.gethostname()=="rc-idsia-macBook.local"
def isserver(): return socket.gethostname()=="758606aabdd8"
def strdate(): return datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
print(socket.gethostname())
!pwd

rc-idsia-macBook.local
/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/papers/pgm20/experiments


In [6]:
if islocal():
    prj_path = "/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/"
    os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home"
elif isserver():
    prj_path="/home/rcabanas/crema/CreMA/"
else:
    warnings.warn("Unknown server, set prj_path variable manually.")
os.chdir(prj_path)
!pwd

/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici


In [31]:
exp_folder = f"{prj_path}/papers/pgm20/experiments/"
log_folder = f"{exp_folder}/logs/"
res_folder = f"{exp_folder}/results/"
jarfile=f"{prj_path}/target/credici-0.1.1-jar-with-dependencies.jar"

rebuild = True

In [19]:
if rebuild:
    !mvn clean compile assembly:single

[INFO] Scanning for projects...
[INFO] 
[INFO] --------------------------< ch.idsia:credici >--------------------------
[INFO] Building credici 0.1.1
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ credici ---
[INFO] Deleting /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/target
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ credici ---
[WARNING] Using platform encoding (US-ASCII actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] Copying 0 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ credici ---
[INFO] Changes detected - recompiling the module!
[WARNING] File encoding has not been set, using platform encoding US-ASCII, i.e. build is platform dependent!
[INFO] Compiling 24 source files to /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/target/classes
[WARNING] /Users/rcabanas/Goo

In [32]:
disable_java = False
@timeout_decorator.timeout(7*6*60)
def run_java(jarfile, javafile):
    cmd=f"java -cp {jarfile} {javafile}"
    if disable_java: cmd= "echo 0,0,0,0,0,0"    
    print(cmd)
    result =subprocess.run(cmd, universal_newlines = True, shell=True,stdout = subprocess.PIPE)
    print(result.stdout)
    return result

# ChainNonMarkovian 6 5 1 -1 0 CCALP 1234
def run_chain(model, N, endovarsize, exovarsize, target, obsvar, dovar, method, seed):
    
    
    print(strdate())
    exovarsize = exovarsize or endovarsize*endovarsize + 1
    
    if obsvar is None: obsvar = -1;
    elif obsvar<0: obsvar = N + obsvar;
    
    if dovar is None: dovar = -1;    
    elif dovar<0: dovar = N + dovar;
    
    if target is None: target = N//2;
    elif target<0: target = N + target;
    
    warmups = 2
    repetitions = 5
    eps = 0.0001
    #ChainNonMarkovian 4  -v 5 -V 9 -t 1 -o -1 -d 0 -m  CCALP --seed 12234 --warmpus 0 --repetitions 1
    javafile = f"{exp_folder}/RunExperiments.java {model} {N} -v {endovarsize} -V {exovarsize} -t {target} -o {obsvar} -d {dovar} -m {method} -e {eps} -s {seed} -w {warmups} -r {repetitions} "  
    try:
        result = run_java(jarfile, javafile)
        output = [float(x) for x in result.stdout.splitlines()[-1].split(",")]
    except:
        output = [float("inf"),float("inf")]+[float("nan")]* (endovarsize*2)
        
    return output

def run_chain_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("ChainMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_chain_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("ChainNonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)


def run_hmm_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("HMM-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_hmm_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("HMM-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)


def run_rhmm_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("RHMM-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_rhmm_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("RHMM-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_squares_markovian(N=4, endovarsize=2, exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("Squares-Markovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_squares_nonmarkovian(N=4, endovarsize=2,exovarsize=None, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("Squares-NonMarkovian", N, endovarsize, exovarsize, target, obsvar, dovar, method, seed)

def run_chain_terbin_markovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):       
    return run_chain("TerBinChainMarkovian", N, 2, None, target, obsvar, dovar, method, seed)

def run_chain_terbin_nonmarkovian(N=4, target=None, obsvar=-1, dovar=0, method="CVE", seed=1234):
    return run_chain("TerBinChainNonMarkovian", N, 2, None, target, obsvar, dovar, method, seed)

In [33]:
!java -cp /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici//target/credici-0.1.1-jar-with-dependencies.jar /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici//papers/pgm20/experiments//RunExperiments.java ChainMarkovian 3 -v 2 -V 6 -t 1 -o 2 -d 0 -m CVE -e 0.0001 -s 1234 -w 2 -r 5 

#!ls /Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici//target/credici-0.1.1-jar-with-dependencies.jar




ChainMarkovian
   N=3 endovarsize=2 exovarsize=6 target=1 obsvar=2 dovar=0 method=CVE seed=1234
max heap memory = -Xmx4g -Xmx4096m
Running experiments...
Warm-up #0 in 63.167 ms.
Warm-up #1 in 2.978 ms.
Measurement #0 in 3.257 ms.
Measurement #1 in 3.316 ms.
Measurement #2 in 2.821 ms.
Measurement #3 in 2.659 ms.
Measurement #4 in 2.46 ms.
[0.9758576874205844, 0.9758576874205844]
2.9026000000000005,2.3644,0.9758576874205844,0.9758576874205844,0.0241423125794155,0.0241423125794155


In [34]:

def run_experiments(f, args, outkeys, fargs=None, verbose=False, lenght_dep_vars = None, non_evaluable=[]):
     
    print("=========")
    print(args)
    print("=========")         
        
    result = pd.DataFrame(columns=list(args.keys())+list(outkeys))
    log_file = f"{log_folder}{strdate()}_{f.__name__}.txt"
    
    data = pd.DataFrame(list(itertools.product(*list(args.values()))), columns = args.keys())
    
    fargs = fargs or {}
    for k,v in fargs.items():
        data[k]=data.apply(v, axis=1)
        
        
    non_evaluable = non_evaluable or [] 
    
    lenght_dep_vars = lenght_dep_vars or ["N"]
    
    def is_evaluable(args):
        current = {k:v for (k,v) in args.items() if k not in lenght_dep_vars}
        previous = [{k:v for (k,v) in a.items() if k not in lenght_dep_vars} for a in non_evaluable]
        
        print(f"current: {current}")
        print(f"previous: {previous}")

        
        return not current in previous
       
    def single_experiment(argsv):
        if is_evaluable(argsv):
            outvals = f(**argsv)
            if np.isnan(outvals).any(): 
                non_evaluable.append(argsv)
                print(f"setting as not evaluable: {argsv}")
        else:
            outvals=[float("nan")]*len(outkeys)
        return outvals    
        
    
    for argsv in data.to_dict(orient="row"):    
        
        strio = io.StringIO()
        
           
        
        with open(log_file, 'a+') as logger:
            if verbose == False:
                with redirect_stdout(strio):
                    outvals = single_experiment(argsv)
            else:
                outvals = single_experiment(argsv)   
            
            logger.write(strio.getvalue())
            
            
        result = result.append({**argsv, **dict(zip(list(outkeys), outvals))}, ignore_index=True)
        print(dict(result.iloc[-1]))
        print("\n\n")
    
    return result

def get_args(**kwargs): return kwargs

##




In [35]:
SEED = [1234+i*1234 for i in range(10)]
res = {}

Note that all the cells below are disabled, just change the `if` condition in the desiered block of experiments 

In [36]:
# with evidence chain markovian  RUN
if True:
    endovarsize = 2
    args = dict(N=list(range(3,20,1)), dovar=[0], obsvar=[-1], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])

    results1 = run_experiments(run_chain_markovian, args, outkeys)
    res_file = f"{res_folder}{strdate()}_chain_markovian_ev.csv"
    results1.to_csv(res_file)

    res["chain_markovian_ev"] = results1

{'N': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'dovar': [0], 'obsvar': [-1], 'seed': [1234, 2468, 3702, 4936, 6170, 7404, 8638, 9872, 11106, 12340], 'exovarsize': [6], 'method': ['CVE', 'CCVE', 'CCALPeps']}
{'N': 3, 'dovar': 0, 'obsvar': -1, 'seed': 1234, 'exovarsize': 6, 'method': 'CVE', 'time': 2.7918, 'query_time': 2.2724, 'lowerbound0': 0.9758576874205844, 'upperbound0': 0.9758576874205844, 'lowerbound1': 0.0241423125794155, 'upperbound1': 0.0241423125794155}



{'N': 3, 'dovar': 0, 'obsvar': -1, 'seed': 1234, 'exovarsize': 6, 'method': 'CCVE', 'time': inf, 'query_time': inf, 'lowerbound0': nan, 'upperbound0': nan, 'lowerbound1': nan, 'upperbound1': nan}



{'N': 3, 'dovar': 0, 'obsvar': -1, 'seed': 1234, 'exovarsize': 6, 'method': 'CCALPeps', 'time': nan, 'query_time': nan, 'lowerbound0': nan, 'upperbound0': nan, 'lowerbound1': nan, 'upperbound1': nan}



{'N': 3, 'dovar': 0, 'obsvar': -1, 'seed': 2468, 'exovarsize': 6, 'method': 'CVE', 'time': inf, 'query_ti

In [ ]:
## with evidence chain NON markovian 
if False:
    endovarsize = 2    
    args = dict(N=list(range(3,25,1)), dovar=[0], obsvar=[-1], seed=SEED, exovarsize=[6], method=["CVE", "CCVE", "CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])

    results1 = run_experiments(run_chain_nonmarkovian, args, outkeys)
    res_file = f"{res_folder}{strdate()}_chain_nonmarkovian_ev.csv"
    results1.to_csv(res_file)

    res["chain_nonmarkovian_ev"] = results1

In [ ]:
# with evidence HMM markovian 

if False:
    endovarsize = 2
    args = dict(N=list(range(3,30,1)), dovar=[0], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 1))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_hmm_markovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_hmm_markovian_ev.csv"
    results1.to_csv(res_file)

    res["hmm_markovian_ev"] = results1

In [ ]:
# with evidence HMM nonmarkovian 

if False:
    endovarsize = 2
    args = dict(N=list(range(3,30,1)), dovar=[0], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 1))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_hmm_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_hmm_nonmarkovian_ev.csv"
    results1.to_csv(res_file)

    res["hmm_nonmarkovian_ev"] = results1

In [ ]:
# with evidence RevHMM nonmarkovian RUN
if False:
    N = list(range(3,15)) + [18,21,24]
    endovarsize = 2
    args = dict(N=N, dovar=[0], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 2))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_rhmm_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_rhmm_nonmarkovian_ev.csv"
    results1.to_csv(res_file)

    res["rhmm_nonmarkovian_ev"] = results1

In [ ]:
# with evidence RevHMM markovian

if False:
    
    endovarsize = 2
    args = dict(N=list(range(3,20,1)), dovar=[0], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 2))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_rhmm_markovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_rhmm_markovian_ev.csv"
    results1.to_csv(res_file)

    res["rhmm_markovian_ev"] = results1

In [ ]:

# with evidence Squares nonmarkovian 

if False:
    endovarsize = 2
    args = dict(N=list(range(3,15,1)), dovar=[0], seed=SEED, exovarsize=[6], method=["CVE", "CCVE","CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 1))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_squares_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars)
    res_file = f"{res_folder}{strdate()}_squares_nonmarkovian_ev.csv"
    results1.to_csv(res_file)

    res["squares_nonmarkovian_ev"] = results1

In [ ]:

# with evidence Squares nonmarkovian 

if False:

    non_evaluable=[{'dovar': 0, 'seed': 3702, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 7404, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 9872, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 11106, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 12340, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 2468, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 4936, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 6170, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 1234, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 8638, 'exovarsize': 6, 'method': 'CCVE'}, {'dovar': 0, 'seed': 1234, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 2468, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 3702, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 4936, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 6170, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 7404, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 8638, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 9872, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 11106, 'exovarsize': 6, 'method': 'CVE'}, {'dovar': 0, 'seed': 12340, 'exovarsize': 6, 'method': 'CVE'}]

    endovarsize = 2
    args = dict(N=[19,20,21], dovar=[0], seed=SEED, exovarsize=[6], method=["CCALPeps"])
    outkeys = ["time", "query_time"]+ sum([[f"lowerbound{i}", f"upperbound{i}"] for i in range(0,endovarsize)], [])
    fargs = dict(target = lambda t: int(((t["N"]-1)//2)*2),
                 obsvar = lambda t: int(t["N"]*2 - 1))

    lenght_dep_vars = ["N", "target", "obsvar"]

    results1 = run_experiments(run_squares_nonmarkovian, args, outkeys, fargs, lenght_dep_vars=lenght_dep_vars, non_evaluable=non_evaluable)
    res_file = f"{res_folder}{strdate()}_squares_nonmarkovian_ev.csv"
    results1.to_csv(res_file)

    res["squares_nonmarkovian_ev"] = results1
